In [0]:
import requests as req
import pandas as pd
import json
from pyspark.sql import functions as f
from pyspark.sql import types as t
import base64

In [0]:
context = json.loads(
    dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()
)
instancename = context["tags"]["browserHostName"]
token = (
    dbutils.widgets._entry_point.getDbutils().notebook().getContext().apiToken().get()
)

In [0]:
job_lst = []

headers = {
    "Authorization": f"Bearer {token}",
}
query_params = {"limit": 25, "offset": 0, "expand_tasks": "true"}
response = req.get(
    f"https://{instancename}/api/2.1/jobs/list", headers=headers, params=query_params
).json()

has_more = response["has_more"]

job_lst = job_lst + response["jobs"]

while has_more:
    query_params["offset"] = query_params["offset"] + query_params["limit"]
    recurse_response = req.get(
        f"https://{instancename}/api/2.1/jobs/list",
        headers=headers,
        params=query_params,
    ).json()
    job_lst = job_lst + recurse_response["jobs"]
    has_more = recurse_response["has_more"]
print("Job List count:", len(job_lst))

Job List count: 509


In [0]:
job_lst

Out[4]: [{'job_id': 669557780682095,
  'creator_user_name': 'ali.azzouz@databricks.com',
  'settings': {'name': 'Jobs-Lab-92-ali.azzouz@databricks.com',
   'email_notifications': {'no_alert_for_skipped_runs': False},
   'timeout_seconds': 0,
   'max_concurrent_runs': 1,
   'tasks': [{'task_key': 'Batch-Job',
     'notebook_task': {'notebook_path': '/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job',
      'source': 'WORKSPACE'},
     'existing_cluster_id': '1005-092209-o2n9ei7m',
     'timeout_seconds': 0,
     'email_notifications': {}},
    {'task_key': 'DLT',
     'depends_on': [{'task_key': 'Batch-Job'}],
     'pipeline_task': {'pipeline_id': '4236927f-46b9-4c2b-aa56-9d35163983b5',
      'full_refresh': False},
     'timeout_seconds': 0,
     'email_notifications': {}},
    {'task_key': 'Query-Results',
     'depends_on': [{'task_key': 'DLT'}],
     'notebook_task': {'notebook_path': 

In [0]:
def get_notebook_path(dic_obj):
    if "notebook_task" in dic_obj.keys():
        notebook_path = dic_obj["notebook_task"].get("notebook_path")
        source = dic_obj["notebook_task"].get("source")
        return {"notebook_path": notebook_path, "source": source}
    else:
        return {
            "notebook_path": "No notebooks tasks here",
            "source": "No notebooks tasks here",
        }

In [0]:
parsed_job_lst = [
    (
        i.get("creator_user_name", "Not available"),
        i["job_id"],
        any([True for data in i["settings"]["tasks"] if "sql_task" in data.keys()]),
        [
            get_notebook_path(data)
            for data in i["settings"]["tasks"]
            if "notebook_task" in data.keys()
        ],
    )
    for i in job_lst
]

In [0]:
parsed_job_lst

Out[7]: [('ali.azzouz@databricks.com',
  669557780682095,
  False,
  [{'notebook_path': '/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job',
    'source': 'WORKSPACE'},
   {'notebook_path': '/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.4L - Query Results Job',
    'source': 'WORKSPACE'}]),
 ('quentin.ambard@databricks.com',
  307420950126174,
  False,
  [{'notebook_path': '/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/_resources/00-prep-data-db-sql',
    'source': 'WORKSPACE'},
   {'notebook_path': '/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/04-Data-Science-ML/04.1-automl-churn-prediction',
    'source': 'WORKSPACE'}]),
 ('prakhar.dixit@databricks.com', 702831209690361, True, []),
 ('abhimanyu.mitra@databricks.com', 700054708492984, False, [])

In [0]:
# data_schema = t.StructType([
#   t.StructField('creator_user_name',t.StringType()),
#   t.StructField('job_id',t.LongType()),
#   t.StructField('contains_sql_task',t.BooleanType()),
#   t.StructField('notebook_tasks',t.ArrayType(t.MapType(t.StringType(),t.StringType())))
# ])
data_schema = t.StructType(
    [
        t.StructField("creator_user_name", t.StringType()),
        t.StructField("job_id", t.LongType()),
        t.StructField("contains_sql_task", t.BooleanType()),
        t.StructField(
            "notebook_tasks",
            t.ArrayType(
                t.StructType(
                    [
                        t.StructField("notebook_path", t.StringType()),
                        t.StructField("source", t.StringType()),
                    ]
                )
            ),
        ),
    ]
)
job_data = spark.createDataFrame(parsed_job_lst, data_schema)

In [0]:
job_data.display()

creator_user_name,job_id,contains_sql_task,notebook_tasks
ali.azzouz@databricks.com,669557780682095,false,"List(List(/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job, WORKSPACE), List(/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.4L - Query Results Job, WORKSPACE))"
quentin.ambard@databricks.com,307420950126174,false,"List(List(/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/_resources/00-prep-data-db-sql, WORKSPACE), List(/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/04-Data-Science-ML/04.1-automl-churn-prediction, WORKSPACE))"
prakhar.dixit@databricks.com,702831209690361,true,List()
abhimanyu.mitra@databricks.com,700054708492984,false,List()
abhimanyu.mitra@databricks.com,65898553210341,false,List()
jaiwant.jonathan@databricks.com,695051168593715,false,"List(List(/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/12 - Productionalizing Dashboards and Queries in DBSQL/DE 12.2L - OPTIONAL Capstone/DE 12.2.3L - Land New Data, WORKSPACE))"
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,"List(List(/Demo/Scalable-ML-3.6.0-IL/Version-3.6.0-IL, WORKSPACE), List(/field-demos/e2-demo-gtm-insurance/notebooks/dlt/02-Curation, WORKSPACE))"
jaiwant.jonathan@databricks.com,170452417297472,false,"List(List(/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job, WORKSPACE), List(/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.4L - Query Results Job, WORKSPACE))"
jaiwant.jonathan@databricks.com,342865836520870,false,"List(List(/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/09 - Task Orchestration with Jobs/DE 9.1 - Scheduling Tasks with the Jobs UI/DE 9.1.2 - Reset, WORKSPACE))"
manish.bharti@databricks.com,45878736681749,false,"List(List(/Users/manish.bharti@databricks.com/Introduction-to-Workflows-v1.0.0/Task-1-Create-Tables, WORKSPACE), List(/Users/manish.bharti@databricks.com/Introduction-to-Workflows-v1.0.0/Task-2-Customers-Data, WORKSPACE), List(/Users/manish.bharti@databricks.com/Introduction-to-Workflows-v1.0.0/Task-3-Products-Data, WORKSPACE), List(/Users/manish.bharti@databricks.com/Introduction-to-Workflows-v1.0.0/Task-4-Orders-Data, WORKSPACE), List(/Users/manish.bharti@databricks.com/Introduction-to-Workflows-v1.0.0/Task-5-Highest-Paid-Customer, WORKSPACE), List(/Users/manish.bharti@databricks.com/Introduction-to-Workflows-v1.0.0/Task-6-Final-Step, WORKSPACE))"


In [0]:
exploded_job_data = job_data.withColumn(
    "notebook_tasks", f.explode_outer("notebook_tasks")
)
cleaned_job_data = exploded_job_data.select(
    "creator_user_name",
    "job_id",
    "contains_sql_task",
    f.col("notebook_tasks.notebook_path").alias("notebook_path"),
    f.col("notebook_tasks.source").alias("source"),
)

In [0]:
exploded_job_data.display()
cleaned_job_data.display()

creator_user_name,job_id,contains_sql_task,notebook_tasks
ali.azzouz@databricks.com,669557780682095,false,"List(/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job, WORKSPACE)"
ali.azzouz@databricks.com,669557780682095,false,"List(/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.4L - Query Results Job, WORKSPACE)"
quentin.ambard@databricks.com,307420950126174,false,"List(/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/_resources/00-prep-data-db-sql, WORKSPACE)"
quentin.ambard@databricks.com,307420950126174,false,"List(/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/04-Data-Science-ML/04.1-automl-churn-prediction, WORKSPACE)"
prakhar.dixit@databricks.com,702831209690361,true,null
abhimanyu.mitra@databricks.com,700054708492984,false,null
abhimanyu.mitra@databricks.com,65898553210341,false,null
jaiwant.jonathan@databricks.com,695051168593715,false,"List(/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/12 - Productionalizing Dashboards and Queries in DBSQL/DE 12.2L - OPTIONAL Capstone/DE 12.2.3L - Land New Data, WORKSPACE)"
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,"List(/Demo/Scalable-ML-3.6.0-IL/Version-3.6.0-IL, WORKSPACE)"
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,"List(/field-demos/e2-demo-gtm-insurance/notebooks/dlt/02-Curation, WORKSPACE)"


creator_user_name,job_id,contains_sql_task,notebook_path,source
ali.azzouz@databricks.com,669557780682095,false,/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job,WORKSPACE
ali.azzouz@databricks.com,669557780682095,false,/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.4L - Query Results Job,WORKSPACE
quentin.ambard@databricks.com,307420950126174,false,/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/_resources/00-prep-data-db-sql,WORKSPACE
quentin.ambard@databricks.com,307420950126174,false,/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/04-Data-Science-ML/04.1-automl-churn-prediction,WORKSPACE
prakhar.dixit@databricks.com,702831209690361,true,null,null
abhimanyu.mitra@databricks.com,700054708492984,false,null,null
abhimanyu.mitra@databricks.com,65898553210341,false,null,null
jaiwant.jonathan@databricks.com,695051168593715,false,/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/12 - Productionalizing Dashboards and Queries in DBSQL/DE 12.2L - OPTIONAL Capstone/DE 12.2.3L - Land New Data,WORKSPACE
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,/Demo/Scalable-ML-3.6.0-IL/Version-3.6.0-IL,WORKSPACE
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,/field-demos/e2-demo-gtm-insurance/notebooks/dlt/02-Curation,WORKSPACE


In [0]:
@f.udf("string")
def workload_type(notebook_path, source):
    lang = None
    if source == "WORKSPACE":
        auth_header = {
            "Authorization": f"Bearer {token}",
        }
        req_data = {"path": notebook_path}
        get_resp = req.get(
            f"https://{instancename}/api/2.0/workspace/list",
            headers=auth_header,
            params=req_data,
        ).json()
        if "objects" in get_resp.keys():
            lang = get_resp["objects"][0].get("language")
        else:
            lang = "Resource unavailable"
    else:
        lang = "Remote Notebook so language unidentifiable"

    return lang

In [0]:
@f.udf(
    t.StructType(
        [
            t.StructField("sql_count", t.IntegerType()),
            t.StructField("sql_percent", t.DoubleType()),
        ]
    )
)
def sql_count_and_percent(notebook_path, source):
    sql_cell_count, sql_percentage = 0, 0
    if source == "WORKSPACE":
        auth_header = {
            "Authorization": f"Bearer {token}",
        }
        data = {"path": notebook_path, "format": "JUPYTER", "direct_download": False}
        get_resp = req.get(
            f"https://{instancename}/api/2.0/workspace/export",
            headers=headers,
            json=data,
        ).json()
        if "content" in get_resp.keys():
            base64_string = get_resp["content"]
            base64_bytes = base64_string.encode("utf-8")
            nb_string_bytes = base64.b64decode(base64_bytes)
            nb_string = nb_string_bytes.decode("utf-8")
            nb_json = json.loads(nb_string)
            for i in nb_json["cells"]:
                if "%sql" in i["source"][0]:
                    sql_cell_count += 1
            sql_percentage = round((sql_cell_count / len(nb_json["cells"]))*100,2)
        else:
            sql_cell_count, sql_percentage = None, None
    else:
        sql_cell_count, sql_percentage = None, None
    return {"sql_count": sql_cell_count, "sql_percent": sql_percentage}

In [0]:
final_job_data = cleaned_job_data.withColumn(
    "lang", workload_type("notebook_path", "source")
).withColumn("sql_stats", sql_count_and_percent("notebook_path", "source"))
final_job_data = final_job_data.withColumn(
    "notebook_workload",
    f.struct(
        "notebook_path",
        "source",
        "lang",
        f.col("sql_stats.sql_count"),
        f.col("sql_stats.sql_percent"),
    ),
).drop("notebook_path", "source", "lang", "sql_stats")
final_job_data.display()

creator_user_name,job_id,contains_sql_task,notebook_workload
ali.azzouz@databricks.com,669557780682095,false,"List(/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.2L - Batch Job, WORKSPACE, PYTHON, 0, 0.0)"
ali.azzouz@databricks.com,669557780682095,false,"List(/Repos/ali.azzouz@databricks.com/data-engineering-with-databricks-english/09 - Task Orchestration with Jobs/DE 9.2L - Jobs Lab/DE 9.2.4L - Query Results Job, WORKSPACE, PYTHON, 2, 16.67)"
quentin.ambard@databricks.com,307420950126174,false,"List(/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/_resources/00-prep-data-db-sql, WORKSPACE, PYTHON, 1, 11.11)"
quentin.ambard@databricks.com,307420950126174,false,"List(/Users/quentin.ambard@databricks.com/test_install2/lakehouse-retail-churn/04-Data-Science-ML/04.1-automl-churn-prediction, WORKSPACE, PYTHON, 0, 0.0)"
prakhar.dixit@databricks.com,702831209690361,true,"List(null, null, Remote Notebook so language unidentifiable, null, null)"
abhimanyu.mitra@databricks.com,700054708492984,false,"List(null, null, Remote Notebook so language unidentifiable, null, null)"
abhimanyu.mitra@databricks.com,65898553210341,false,"List(null, null, Remote Notebook so language unidentifiable, null, null)"
jaiwant.jonathan@databricks.com,695051168593715,false,"List(/Repos/jaiwant.jonathan@databricks.com/Jai-DEwith_databricks/12 - Productionalizing Dashboards and Queries in DBSQL/DE 12.2L - OPTIONAL Capstone/DE 12.2.3L - Land New Data, WORKSPACE, PYTHON, 0, 0.0)"
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,"List(/Demo/Scalable-ML-3.6.0-IL/Version-3.6.0-IL, WORKSPACE, SCALA, null, null)"
harikrishnan.kunhumveettil@databricks.com,445063184107737,false,"List(/field-demos/e2-demo-gtm-insurance/notebooks/dlt/02-Curation, WORKSPACE, PYTHON, 0, 0.0)"


In [0]:
final_job_data.filter(
    "notebook_workload.lang = 'SQL' or contains_sql_task = true"
).groupBy("creator_user_name", "job_id", "contains_sql_task").agg(
    f.collect_list("notebook_workload").alias("notebook_workloads")
).display()

creator_user_name,job_id,contains_sql_task,notebook_workloads
prerna.sharma@databricks.com,198763332969994,false,"List(List(/Users/prerna.sharma@databricks.com/DNB Aug26/01 - Databricks Workspace and Services/Notebook basics_2, WORKSPACE, SQL, null, null))"
nathan.tong@databricks.com,90185521908451,false,"List(List(/Users/nathan.tong@databricks.com/Demos/Delta SQL Rapid Start, WORKSPACE, SQL, null, null), List(/Users/nathan.tong@databricks.com/ntong_test, WORKSPACE, SQL, null, null))"
shashank.kava@databricks.com,963435679477698,false,"List(List(/Repos/shashank.kava@databricks.com/wflowtest/Click/Match, WORKSPACE, SQL, null, null))"
nikit.shah@databricks.com,154471483561685,false,"List(List(/Shared/Tech-Summit-2022/01-Hands-On-Notebooks/1a-DLT-Loan-pipeline-SQL, WORKSPACE, SQL, null, null))"
michael.cranford@databricks.com,117909048792713,false,"List(List(/Users/michael.cranford@databricks.com/dlt demo, WORKSPACE, SQL, null, null), List(/Users/michael.cranford@databricks.com/Quickstart Notebook (1), WORKSPACE, SQL, null, null))"
sheetal.kushwaha@databricks.com,171379905059617,true,"List(List(null, null, Remote Notebook so language unidentifiable, null, null))"
sergio.schena@databricks.com,1079514002898377,true,"List(List(/Users/sergio.schena@databricks.com/Rapid Starts/Intro to Databricks/Workflows/0-Setup, WORKSPACE, SQL, null, null), List(/Users/sergio.schena@databricks.com/Rapid Starts/Intro to Databricks/Workflows/1A - Gym Data, WORKSPACE, SQL, null, null), List(/Users/sergio.schena@databricks.com/Rapid Starts/Intro to Databricks/Workflows/1B - Retail Data, WORKSPACE, SQL, null, null))"
sheetal.kushwaha@databricks.com,224807062860319,true,"List(List(null, null, Remote Notebook so language unidentifiable, null, null))"
tj.cycyota@databricks.com,814426410866895,true,"List(List(/Table-ACL-Backup, WORKSPACE, PYTHON, 0, 0.0), List(/SYNC Alternative, WORKSPACE, SCALA, null, null))"
prakhar.dixit@databricks.com,702831209690361,true,"List(List(null, null, Remote Notebook so language unidentifiable, null, null))"


In [0]:
final_job_data.orderBy(
    f.col("notebook_workload.sql_count").desc(),
    f.col("notebook_workload.sql_percent").desc(),
).groupBy("creator_user_name", "job_id", "contains_sql_task").agg(
    f.collect_list("notebook_workload").alias("notebook_workloads")
).display()

creator_user_name,job_id,contains_sql_task,notebook_workloads
9f48a552-0bb2-4274-936d-cb45bcde8017,149732423563518,false,"List(List(/Users/shashank.kava@databricks.com/01-General_RapidStart/cs-rapidstart, WORKSPACE, PYTHON, 10, 23.26))"
Not available,18831,false,"List(List(/Users/ryan.nagy@databricks.com/Demo/git_demo, WORKSPACE, PYTHON, 0, 0.0))"
Not available,25041,false,"List(List(/Users/cosmin.lita@databricks.com/delta-lake-hands-on/Python/00_ingest_raw, WORKSPACE, PYTHON, 0, 0.0))"
aakanksha.mishra@databricks.com,214263291046825,false,"List(List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-1-Create-Tables, WORKSPACE, PYTHON, 4, 33.33), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-5-Highest-Paid-Customer, WORKSPACE, PYTHON, 3, 30.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-2-Customers-Data, WORKSPACE, PYTHON, 2, 25.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-3-Products-Data, WORKSPACE, PYTHON, 2, 25.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-4-Orders-Data, WORKSPACE, PYTHON, 2, 25.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-6-Final-Step, WORKSPACE, PYTHON, 0, 0.0))"
aakanksha.mishra@databricks.com,465052419762416,false,"List(List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-1-Create-Tables, WORKSPACE, PYTHON, 4, 33.33), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-2-Customers-Data, WORKSPACE, PYTHON, 2, 25.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-3-Products-Data, WORKSPACE, PYTHON, 2, 25.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-4-Orders-Data, WORKSPACE, PYTHON, 2, 25.0))"
aakanksha.mishra@databricks.com,552501153966481,false,"List(List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-1-Create-Tables, WORKSPACE, PYTHON, 4, 33.33))"
aakanksha.mishra@databricks.com,854324939393044,false,"List(List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-1-Create-Tables, WORKSPACE, PYTHON, 4, 33.33), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-2-Customers-Data, WORKSPACE, PYTHON, 2, 25.0), List(/Users/aakanksha.mishra@databricks.com/Introduction-to-Workflows-v1.0.0/Task-3-Products-Data, WORKSPACE, PYTHON, 2, 25.0))"
aakanksha.mishra@databricks.com,977293901160877,false,"List(List(/Users/aakanksha.mishra@databricks.com/Retrieve baby names, WORKSPACE, PYTHON, 0, 0.0), List(/Users/aakanksha.mishra@databricks.com/Filter baby names, WORKSPACE, PYTHON, 0, 0.0))"
abhimanyu.mitra@databricks.com,17861092837509,false,"List(List(/Users/abhimanyu.mitra@databricks.com/Enablement/Multi Task Jobs/task-0, WORKSPACE, PYTHON, 0, 0.0), List(/Users/abhimanyu.mitra@databricks.com/Enablement/Multi Task Jobs/task-2, WORKSPACE, PYTHON, 0, 0.0), List(/Users/abhimanyu.mitra@databricks.com/Enablement/Multi Task Jobs/task-1, WORKSPACE, PYTHON, 0, 0.0), List(/Users/abhimanyu.mitra@databricks.com/Enablement/Multi Task Jobs/task-0, WORKSPACE, PYTHON, 0, 0.0))"
abhimanyu.mitra@databricks.com,65898553210341,false,"List(List(null, null, Remote Notebook so language unidentifiable, null, null))"


Repos: `file:/Workspace/Repos`

DBFS: `file:/dbfs/FileStore`